In [ ]:
import numpy as np
import pandas as pd
from sklearn.linear_model import LinearRegression
import matplotlib.pyplot as plt
import matplotlib.patches as mpatches
import seaborn as sns
import itertools
import warnings

%matplotlib inline

In [ ]:
pd.set_option('display.max_rows', 700)
warnings.filterwarnings('ignore')

In [ ]:
def get_data():
    
    wc = pd.read_csv('data/results.csv')
    
    fifa = pd.read_csv('data/fifa_ranking.csv')
    
    return wc, fifa

In [ ]:
def wrangle(wc, fifa):
    
    wc['date'] = pd.to_datetime(wc['date'], infer_datetime_format=True)
    wc.rename(columns={'country': 'host_country', 'neutral': 'neutral_site'}, inplace=True)
    wc = wc[wc['tournament'].str.contains('FIFA')]
    wc = wc[(wc['date'].dt.year >= 1995) & (wc['date'].dt.year <= 2018)]
    wc.reset_index(drop=True, inplace=True)
    
    # Country names by FIFA:
    # Brunei = Brunei Darussalam
    # Burma = Myanmar
    # Cape Verde = Cape Verde Islands
    # DR Congo = Congo DR
    # East Timor = Timor-Leste
    # Eswatini = Swaziland
    # Iran = IR Iran
    # Ivory Coast = Côte d'Ivoire
    # Kyrgyzstan = Kyrgyz Republic
    # North Korea = Korea DPR
    # North Macedonia = FYR Macedonia
    # Saint Kitts and Nevis = St Kitts and Nevis
    # Saint Lucia = St Lucia
    # Saint Vincent and the Grenadines = St Vincent and the Grenadines
    # South Korea = Korea Republic
    # São Tomé and Príncipe = São Tomé e Príncipe
    # U.S. Virgin Islands = US Virgin Islands
    # United States = USA

    fifa_names = ['USA', 'US Virgin Islands', 'São Tomé e Príncipe', 'Korea Republic', 'St Vincent and the Grenadines',\
                 'St Lucia', 'St Kitts and Nevis', 'FYR Macedonia', 'Korea DPR', 'Kyrgyz Republic', "Côte d'Ivoire",\
                 'IR Iran', 'Swaziland', 'Timor-Leste', 'Congo DR', 'Cape Verde Islands', 'Myanmar', 'Brunei Darussalam']
    
    wc_names = ['United States', 'U.S. Virgin Islands', 'São Tomé and Príncipe', 'South Korea',\
                'Saint Vincent and the Grenadines', 'Saint Lucia', 'Saint Kitts and Nevis', 'North Macedonia',\
                'North Korea', 'Kyrgyzstan', 'Ivory Coast', 'Iran', 'Eswatini', 'East Timor', 'DR Congo',\
                'Cape Verde', 'Burma', 'Brunei']
    
    mapping_fifa = dict(zip(fifa_names, wc_names))
    fifa.loc[fifa['country_full'].isin(fifa_names), 'country_full'] = \
    fifa.loc[fifa['country_full'].isin(fifa_names), 'country_full'].map(mapping_fifa)

    confed = fifa[['country_full', 'confederation']].groupby('country_full').max()
    
    # we're only interested in games related to each WC edition
    years = wc['date'].dt.year.copy()
    wc_years = [1994, 1998, 2002, 2006, 2010, 2014]
    for y in wc_years:
        years.mask((y < years) & (years < y + 4), y + 4, inplace=True)

    # revamping the tournament feature so that each game is labeled according to its WC edition
    wc['tournament'] = (wc['tournament']
                        .map({'FIFA World Cup qualification': 'WC_q', 'FIFA World Cup': 'WC'})
                        .str.cat(years.astype('str'), sep=' '))

    # creating a "stage" feature representing at which stage of the competition the game occurred
    stage = np.empty(0, dtype=object)
    for y in range(6):
        new_stage = np.empty(64, dtype=object)
        new_stage[0:48] = 'G' # group stage
        new_stage[48:56] = 'R16' # round of 16
        new_stage[56:60] = 'QF' # quarter-finals
        new_stage[60:62] = 'SF' # semi-finals
        new_stage[62:63] = 'LF' # losers finals
        new_stage[63:64] = 'WF' # winners finals
        stage = np.concatenate((stage, new_stage), axis = 0)

    wc['stage'] = ['Quals'] * len(wc) # qualifications
    wc_mask = wc['tournament'].str.contains(r'WC \d{4}', regex=True)
    wc.loc[wc_mask, 'stage'] = stage

    # creating a "winner" feature, labeled according to the winner of the game
    wc['winner'] = [''] * len(wc)
    wc['winner'] = ((wc['home_score'] > wc['away_score']) * wc['home_team'] +
                    (wc['home_score'] < wc['away_score']) * wc['away_team'])

    # creating a "agg_score" feature
    wc['tot_goals'] = wc['home_score'] + wc['away_score']

    # most tie games had no winners; only those in the knockout stage of a WC had a winner decided by a penalty shoot-out
    winners_idx = [686, 689, 692, 1512, 1520, 2415, 2418, 2421, 2425, 3276, 
                   3279, 4124, 4126, 4135, 4137, 4988, 4989, 4993, 4997]
    winners = ['Argentina', 'France', 'Brazil', 'Spain', 'South Korea', 'Ukraine', 'Germany', 'Portugal', 'Italy', 'Paraguay', 
               'Uruguay', 'Brazil', 'Costa Rica', 'Netherlands', 'Argentina', 'Russia', 'Croatia', 'England', 'Croatia']
    wc['winner'].iloc[winners_idx] = winners

    # cleaning the tournament feature labels
    wc.loc[wc['tournament'].str.contains(r'WC_q \d{4}', regex=True), 'tournament'] = \
    wc.loc[wc['tournament'].str.contains(r'WC_q \d{4}', regex=True), 'tournament'].str.split('_q').str.join('')

    # dropping unnecessary features and rearranging the dataframe
    wc.drop(['date', 'city'], axis=1, inplace=True)
    wc = wc[['host_country', 'tournament', 'stage', 'home_team', 'home_score', 
             'away_team', 'away_score', 'tot_goals', 'winner', 'neutral_site']]
    
    # appending to the dataframes the confederations to which each team belongs
    countries = wc['home_team'].unique()
    s = confed.loc[countries].sort_index()
    wc['confederation'] = wc['home_team'].map(dict(s['confederation']))
    
    return wc

In [ ]:
def wc_dfs(wc, years=[1998, 2002, 2006, 2010, 2014, 2018], quals=False):
    
    editions = ['1998 WC France', '2002 WC South Korea-Japan', '2006 WC Germany',\
                '2010 WC South Africa', '2014 WC Brazil', '2018 WC Russia']

    # only WC torunament games
    wc_1998_tourn = (wc[(wc['tournament'] == 'WC 1998') & (wc['stage'] != 'Quals')]
                     .reset_index(drop=True)
                     .drop(['host_country', 'tournament'], axis=1))
    wc_2002_tourn = (wc[(wc['tournament'] == 'WC 2002') & (wc['stage'] != 'Quals')]
                     .reset_index(drop=True)
                     .drop(['host_country', 'tournament'], axis=1))
    wc_2006_tourn = (wc[(wc['tournament'] == 'WC 2006') & (wc['stage'] != 'Quals')]
                     .reset_index(drop=True)
                     .drop(['host_country', 'tournament'], axis=1))
    wc_2010_tourn = (wc[(wc['tournament'] == 'WC 2010') & (wc['stage'] != 'Quals')]
                     .reset_index(drop=True)
                     .drop(['host_country', 'tournament'], axis=1))
    wc_2014_tourn = (wc[(wc['tournament'] == 'WC 2014') & (wc['stage'] != 'Quals')]
                     .reset_index(drop=True)
                     .drop(['host_country', 'tournament'], axis=1))
    wc_2018_tourn = (wc[(wc['tournament'] == 'WC 2018') & (wc['stage'] != 'Quals')]
                     .reset_index(drop=True)
                     .drop(['host_country', 'tournament'], axis=1))

    tourns_df = pd.concat([wc_1998_tourn, wc_2002_tourn, wc_2006_tourn,\
                    wc_2010_tourn, wc_2014_tourn, wc_2018_tourn], axis=1, keys=editions)
    
    # only WC qualification games
    wc_1998_quals = (wc[(wc['tournament'] == 'WC 1998') & (wc['stage'] == 'Quals')]
                     .reset_index(drop=True)
                     .drop(['host_country', 'tournament'], axis=1))
    wc_2002_quals = (wc[(wc['tournament'] == 'WC 2002') & (wc['stage'] == 'Quals')]
                     .reset_index(drop=True)
                     .drop(['host_country', 'tournament'], axis=1))
    wc_2006_quals = (wc[(wc['tournament'] == 'WC 2006') & (wc['stage'] == 'Quals')]
                     .reset_index(drop=True)
                     .drop(['host_country', 'tournament'], axis=1))
    wc_2010_quals = (wc[(wc['tournament'] == 'WC 2010') & (wc['stage'] == 'Quals')]
                     .reset_index(drop=True)
                     .drop(['host_country', 'tournament'], axis=1))
    wc_2014_quals = (wc[(wc['tournament'] == 'WC 2014') & (wc['stage'] == 'Quals')]
                     .reset_index(drop=True)
                     .drop(['host_country', 'tournament'], axis=1))
    wc_2018_quals = (wc[(wc['tournament'] == 'WC 2018') & (wc['stage'] == 'Quals')]
                     .reset_index(drop=True)
                     .drop(['host_country', 'tournament'], axis=1))

    quals = [wc_1998_quals, wc_2002_quals, wc_2006_quals, wc_2010_quals, wc_2014_quals, wc_2018_quals]
    quals_dict = dict(zip(editions, quals))
        
    mapping_editions = {1998: '1998 WC France', 2002: '2002 WC South Korea-Japan', 2006: '2006 WC Germany',\
                        2010: '2010 WC South Africa', 2014: '2014 WC Brazil', 2018: '2018 WC Russia'}
    if quals:
        return tourns_df[list(map(lambda y: mapping_editions[y], years))]
    else:
        output = []
        for ed in list(map(lambda y: mapping_editions[y], years)):
            output.append(quals_dict[ed])
        return output

In [ ]:
editions = ['1998 WC France', '2002 WC South Korea-Japan', '2006 WC Germany',\
            '2010 WC South Africa', '2014 WC Brazil', '2018 WC Russia']
    
def ties_per_wc(tourns_df):
    
    draws = {ed: (tourns_df[ed]['home_score'] == tourns_df[ed]['away_score']).sum() for ed in editions}
    df = pd.DataFrame(draws, index=['tie games']).T
    fig, ax = plt.subplots(1, 1, figsize=(9, 6))
    line_chart = sns.pointplot(x=df.index, y='tie games', data=df)
    plt.title('Tie games in regular time', y=1.05, fontsize=20)
    ax.set_ylabel('')
    plt.xticks(rotation='vertical')
    
    return fig
    
def avg_goals_per_wc(tourns_df):
    
    avg_goals = {ed: (tourns_df[ed]['tot_goals']).mean() for ed in editions}
    df = pd.DataFrame(avg_goals, index=['avg goals']).T
    fig, ax = plt.subplots(1, 1, figsize=(9, 6))
    line_chart = sns.pointplot(x=df.index, y='avg goals', data=df)
    plt.title('Average number of goals per game', y=1.05, fontsize=20)
    ax.set_ylabel('')
    plt.xticks(rotation='vertical')
    
    return fig

def wc_tot_goals_per_stage(tourns_df):
    
    df = pd.concat([tourns_df[editions[0]][['stage', 'tot_goals']], tourns_df[editions[1]][['stage', 'tot_goals']],\
                    tourns_df[editions[2]][['stage', 'tot_goals']], tourns_df[editions[3]][['stage', 'tot_goals']],\
                    tourns_df[editions[4]][['stage', 'tot_goals']], tourns_df[editions[5]][['stage', 'tot_goals']]],\
                    axis=0, ignore_index=True)

    df_G = df[df['stage'] =='G']
    df_R16 = df[df['stage'] =='R16']
    df_QF = df[df['stage'] =='QF']
    df_SF = df[df['stage'] =='SF']
    df_LF = df[df['stage'] =='LF']
    df_WF = df[df['stage'] =='WF']

    data = [df_G['tot_goals'], df_R16['tot_goals'], df_QF['tot_goals'], \
            df_SF['tot_goals'], df_LF['tot_goals'], df_WF['tot_goals']]

    fig, ax = plt.subplots(1, 1, figsize=(9, 6))
    plt.title('Goals per stage aggregated over all WCs', y=1.05, fontsize=20)
    boxplots = ax.boxplot(data)
    ax.set_xticklabels(['G', 'R16', 'QF', 'SF', 'LF', 'WF'])
    
    return fig

def avg_goals_per_wc_stage(tourns_df):
    
    stages = ['G', 'R16', 'QF', 'SF', 'LF', 'WF']

    df = pd.concat([tourns_df[editions[0]][['stage', 'tot_goals']], tourns_df[editions[1]][['stage', 'tot_goals']],\
                    tourns_df[editions[2]][['stage', 'tot_goals']], tourns_df[editions[3]][['stage', 'tot_goals']],\
                    tourns_df[editions[4]][['stage', 'tot_goals']], tourns_df[editions[5]][['stage', 'tot_goals']]],\
                    axis=1, keys=editions)

    pairs = list(itertools.product(range(2), range(3)))
    fig, ax = plt.subplots(2, 3, figsize=(12,8), sharex=True, sharey=True)
    fig.suptitle('Average goals per stage', fontsize=20)
    for i, (j, k) in enumerate(pairs):
        ax[j, k].set_title(editions[i])
        sns.pointplot(df[editions[i]]['stage'], df[editions[i]]['tot_goals'], ax=ax[j, k])
        ax[j, k].set_xlabel('')
        ax[j, k].set_ylabel('')
    plt.tight_layout()
    plt.subplots_adjust(top=0.85)
    
    return fig
    
def tot_wins_per_wc(tourns_df):
    
    wins_per_wc = {ed: tourns_df[ed].loc[tourns_df[ed]['winner'] != '', 'winner'].value_counts() for ed in editions}

    for ed in editions:
        teams = set(tourns_df[ed][['home_team', 'away_team']].stack())
        no_wins = list(teams.difference(set(wins_per_wc[ed].index)))
        zeros = [0] * len(no_wins)
        s = pd.Series(zeros, no_wins)
        wins_per_wc[ed] = pd.concat([wins_per_wc[ed], s], axis=0)

    host_country = ['France', 'South Korea-Japan', 'Germany', 'South Africa', 'Brazil', 'Russia']
    colors = [None] * 6
    for idx, ed in enumerate(editions):
        hc = host_country[idx]
        colors[idx] = ['r' if country in hc.split('-') else 'b' for country in wins_per_wc[ed].index]

    wc_champs = [0, 0, 1, 1, 2, 0]

    pairs = list(itertools.product(range(2), range(3)))
    fig, ax = plt.subplots(2, 3, figsize=(16, 10))
    fig.suptitle('Total number of wins', y=1, fontsize=20)
    for i, (j, k) in enumerate(pairs):
        ax[j, k].set_title(editions[i])
        bars = ax[j, k].bar(wins_per_wc[editions[i]].index, wins_per_wc[editions[i]].values, color=colors[i])
        bars[wc_champs[i]].set_edgecolor('y')
        bars[wc_champs[i]].set_linewidth(3)
        for tick in ax[j, k].get_xticklabels():
            tick.set_rotation('vertical')
    red_patch = mpatches.Patch(color='red', label='host country')
    yellow_patch = mpatches.Patch(color='yellow', label='WC champion')
    ax[0, 0].legend(handles=[red_patch, yellow_patch], loc='upper right')
    plt.tight_layout()
    plt.subplots_adjust(top=0.9)
    
    return fig

In [ ]:
def get_plot(df, plot_func):
    return plot_func(df)

def save_viz(plot, title):
    plot.savefig('output/' + title + '.png')

In [ ]:
def feature_engineering(tourns_df, quals_dict, fifa):
    
    editions = ['1998 WC France', '2002 WC South Korea-Japan', '2006 WC Germany',\
                '2010 WC South Africa', '2014 WC Brazil', '2018 WC Russia']
        
    # countries that got direct entry to the WC:
    direct_entry = {'1998 WC France': 
                        {'countries': ['France', 'Brazil'],
                         'confeds': {'France': 'UEFA', 'Brazil': 'CONMEBOL'}},
                    '2002 WC South Korea-Japan':
                        {'countries': ['South Korea', 'Japan', 'France'],
                         'confeds': {'South Korea': 'AFC', 'Japan': 'AFC', 'France': 'UEFA'}},
                    '2006 WC Germany': 
                        {'countries': ['Germany'],
                         'confeds': {'Germany': 'UEFA'}},
                    '2010 WC South Africa':
                        {'countries': ['South Africa'],
                         'confeds': {'South Africa': 'CAF'}}, # yet, they still participated in the 2010 WC CAF qualifiers
                    '2014 WC Brazil':                         # however, this won't be considered, since is an anomaly
                        {'countries': ['Brazil'],
                         'confeds': {'Brazil': 'CONMEBOL'}},
                    '2018 WC Russia':
                        {'countries': ['Russia'],
                         'confeds': {'Russia': 'UEFA'}}}

    # total number of points each team made in the qualification stage to the WC
    quals_info = {}
    for ed in editions:
        teams = sorted(tourns_df[ed][['home_team', 'away_team']].stack().unique())
        quals_info[ed] = {}
        for c in teams:
            df = quals_dict[ed][(quals_dict[ed]['home_team'] == c) | (quals_dict[ed]['away_team'] == c)]
            if c in direct_entry[ed]['countries']:
                qual_pts = np.nan
                confederation = direct_entry[ed]['confeds'][c]
            else:
                qual_pts = df['winner'].tolist().count(c) * 3 + df['winner'].tolist().count('')
                confederation = df['confederation'].iloc[0]
            quals_info[ed][c] = [qual_pts, confederation]
            
    # label encoding the stage feature
    mapping_stage = {'G': 0, 'R16': 1, 'QF': 2, 'SF': 3, 'LF': 4, 'WF': 5}
    df = []
    for ed in editions:
        df.append(tourns_df[ed].join(tourns_df[ed]['stage'].map(mapping_stage), rsuffix='_enc').drop('stage', axis=1))
        
    tourns_df = pd.concat(df, axis=1, keys=editions)

    # furthest stage reached in the WC
    wc_champions = dict(zip(editions, ['France', 'Brazil', 'Italy', 'Spain', 'Germany', 'France']))
    furthest_stage = {}
    for ed in editions:
        furthest_stage[ed] = {}
        for c, v in quals_info[ed].items():
            s = (tourns_df[ed]['home_team'] == c) | (tourns_df[ed]['away_team'] == c)
            furthest_stage[ed][c] = tourns_df[ed].loc[s[s==True].index[-1], 'stage_enc']
            if c == wc_champions[ed]:
                furthest_stage[ed][c] += 1
    
    data = {}
    for ed in editions:
        df_1 = pd.DataFrame(furthest_stage[ed], index=['furthest_stage']).T
        df_2 = pd.DataFrame(quals_info[ed], index=['qual_pts', 'confederation']).T
        data[ed] = pd.concat([df_1, df_2], axis=1)
        
    # a statistic that aims at measuring how good the confederations' teams perform on the WCs on average
    confed_metrics = pd.Series()
    for ed in editions:
        confed_metrics[ed] = data[ed].groupby('confederation')['furthest_stage'].mean()
        data[ed]['confed_metrics'] = data[ed]['confederation'].map(confed_metrics[ed])
        
    # calculating the total number of appearances in WCs up until, and including, that edition
    n_appearances = pd.DataFrame()
    for i, ed in enumerate(editions):
        df = pd.DataFrame(index=data[ed].index, data=[1]*len(data[ed].index), columns=[ed])
        n_appearances = n_appearances.merge(df, left_index=True, right_index=True, how='outer')
        n_appearances.fillna(0, inplace=True)
        try:
            n_appearances[editions[i]] += n_appearances[editions[i - 1]]
        except:
            pass

    for ed in editions:
        data[ed]['n_appearances'] = n_appearances[ed].loc[data[ed].index]
        
    # including an average FIFA ranking feature for teams
    rankings = fifa[['rank', 'country_full', 'rank_date']]
    rankings.columns = ['rank', 'country', 'date']
    rankings['date'] = pd.to_datetime(rankings['date'], infer_datetime_format=True)
    rankings = rankings[rankings['date'] >= '1996-03-10']
    rankings['date'] = rankings['date'].dt.year

    years = rankings['date']
    wc_years = [1994, 1998, 2002, 2006, 2010, 2014]
    for y in wc_years:
        years.mask((y < years) & (years < y + 4), y + 4, inplace=True)

    rank_df = rankings.groupby(['country', 'date']).mean()
    rank_df.reset_index(inplace=True)
    rank_df.sort_values(['country', 'date'], inplace=True)
    rank_df.rename(columns={'rank': 'avg_rank'}, inplace=True)
    rank_df = rank_df[rank_df['country'].isin(n_appearances.index)].reset_index(drop=True)

    rank_df = rank_df.pivot(index='country', columns='date', values='avg_rank')
    rank_df.columns.name = None
    rank_df.index.name = None
    rank_df.columns = ['avg_rank_1998', 'avg_rank_2002', 'avg_rank_2006',\
                       'avg_rank_2010', 'avg_rank_2014', 'avg_rank_2018']
    
    # merging the WC df with the FIFA rankings df for each edition of the WC
    for i, ed in enumerate(editions):
        data[ed] = pd.merge(data[ed], rank_df.iloc[:, 0:i+1], left_index=True, right_index=True, how='left')
        
    # let's fill in the missing values for Serbia with the median of the corresponding confederation
    median_uefa = {}
    for i, ed in enumerate(editions):
        median_uefa[ed] = data[ed][data[ed]['confederation'] == 'UEFA'].iloc[:, 5:6+i].median(axis=0, skipna=True)

    data[editions[0]].loc['Serbia', 'avg_rank_1998'] = median_uefa[editions[0]].values
    data[editions[2]].loc['Serbia', ['avg_rank_1998', 'avg_rank_2002']] =\
    [median_uefa[editions[2]].values[0], median_uefa[editions[2]].values[1]]
    data[editions[3]].loc['Serbia', ['avg_rank_1998', 'avg_rank_2002']] =\
    [median_uefa[editions[3]].values[0], median_uefa[editions[3]].values[1]]
    data[editions[5]].loc['Serbia', ['avg_rank_1998', 'avg_rank_2002']] =\
    [median_uefa[editions[5]].values[0], median_uefa[editions[5]].values[1]]
        
    return data

In [ ]:
def analysis(data, year):
    
    mapping_editions = {1998: '1998 WC France', 2002: '2002 WC South Korea-Japan', 2006: '2006 WC Germany',\
                        2010: '2010 WC South Africa', 2014: '2014 WC Brazil', 2018: '2018 WC Russia'}
    
    ed = mapping_editions[year]
    
    corr_data = data[ed].corr()

    fig, ax = plt.subplots(figsize=(8,8))

    mask = np.zeros_like(corr_data, dtype=np.bool)
    mask[np.triu_indices_from(mask)] = True
    sns.heatmap(corr_data, mask=mask, cmap='Blues', annot=True, square=True, cbar_kws={"shrink": .75})
    ax.set_ylim(len(corr_data), 0, 0)
    
    return fig

In [ ]:
if __name__ == '__main__':
    wc, fifa = get_data()
    wc = wrangle(wc, fifa)
    df = wc_dfs(wc, years=[1998, 2002, 2006, 2010, 2014, 2018], quals=False)
    plot = get_plot(df, plot_func=tot_wins_per_wc)
    ###################################################################################################################
    # DOCUMENTATION FOR get_plot(df, plot_func) FUNCTION:                                                             #
    #     > only for WC tournament games (quals=False)                                                                #
    #     > all WC editions from 1998 to 2018 must be included                                                        #
    #     > plot_func: ties_per_wc, avg_goals_per_wc, wc_tot_goals_per_stage, avg_goals_per_wc_stage, tot_wins_per_wc #
    ###################################################################################################################          
    tourns_df = wc_dfs(wc, years=[1998, 2002, 2006, 2010, 2014, 2018], quals=False)
    quals_dict = wc_dfs(wc, years=[1998, 2002, 2006, 2010, 2014, 2018], quals=True)
    final_df = feature_engineering(tourns_df, quals_dict, fifa)
    results = analysis(final_df, year=2018)
    save_viz(plot, title='plot')